In [2]:
import pandas as pd
import json
pd.set_option('max_colwidth', None)
pd.set_option("max_rows", None)

# pd.set_option('max_rows',20)
# pd.set_option('max_columns',10)
# pd.set_option('max_colwidth', 20)

In [3]:
df=pd.read_json('data_analytics_results.json').transpose()
#df.head()

### Data Cleaning

In [4]:
# cleaning contents
from bs4 import BeautifulSoup
def parse_html_to_text(content):
    return BeautifulSoup(content, features="html.parser").get_text()

df['contents'] = df['contents'].apply(parse_html_to_text)
df['contents'].iloc[0:4]

11414403                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [5]:
# Removing extra characters like '\t','\n' etc
df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

In [6]:
df['publication_date'].str.split('T',expand=True)
df['date_published']=df['publication_date'].str.split('T',expand=True)[0]
df['time_published']=df['publication_date'].str.split('T',expand=True)[1]
df['time_published']=df['time_published'].str.replace('\D$','',regex=True)
df.drop('publication_date',axis=1,inplace=True)

In [7]:
#checking if all the values for df['type'] column is external
print(all(map(lambda x:x=='external',df['type'])))
print(df['model_type'].nunique())

False
1


In [8]:
df['type'].value_counts()

external    7904
native         1
Name: type, dtype: int64

In [9]:
df.set_index('id',inplace=True)
df.drop(['short_name','model_type','categories','type','refs','tags'],axis=1,inplace=True)

In [10]:
def clean_locations(loc):
   locations=[]
   #states=[]
   if loc!=None and len(loc)!=0:
      for i in range(len(loc)):
            locations.append(loc[i]['name'])
      #temp=loc[0]['name'].split(',')[1]
   return locations

In [11]:
df['locations']=df['locations'].apply(clean_locations)
df['levels']=df['levels'].apply(lambda level:level[0]['short_name'])
#df['tags']=df['tags'].apply(lambda tag: tag[0]['name'] if len(tag) != 0 else None)
#df['refs']=df['refs'].apply(lambda ref:ref['landing_page'])
df['company']=df['company'].apply(lambda company:company['name'])

In [12]:
def func_states(loc):
    if loc==None:
        return []
    elif 'Flexible / Remote' in loc:
        return ['Remote']
    else:
        lst=[]
        for i in range(len(loc)):
            if loc[i]!='United States':
                city=loc[i].split(',')[0].strip()
                st=loc[i].split(',')[1].strip()
                lst.append((city,st))
                # if st not in states:
                #     states.append(st)
                # if city not in cities:
                #     cities.append(city)
    return lst



In [13]:
df['locations']=df['locations'].apply(func_states)

In [14]:
# df_new=df[['name', 'states', 'levels', 'tags', 'company',
#        'date_published', 'time_published']].explode('states')
# df_new.reset_index()

In [15]:
df[['name', 'contents', 'levels', 'company',
        'date_published', 'time_published']].to_csv('data_analytics_clean.csv')

In [36]:
locations_df=df[['locations']].explode('locations').reset_index()
locations_df

,id,locations
0,11414403,"(Albuquerque, NM)"
1,11380066,Remote
2,11375065,"(Calhoun, GA)"
3,11291995,"(Charlotte, NC)"
4,11409323,"(Bellevue, WA)"
5,9040728,"(Albuquerque, NM)"
6,11418135,"(Austin, TX)"
7,10917102,"(Dallas, TX)"
8,11214288,"(Bellingham, MA)"
9,11176518,Remote


In [37]:
locations_df.columns

Index(['id', 'locations'], dtype='object')

In [38]:
locations_df['city']=locations_df['locations'].apply(lambda city:city[0] if len(city)==2 else city)

In [39]:
locations_df['state']=locations_df['locations'].apply(lambda city:city[1] if len(city)==2 else city)

In [40]:
locations_df.head()

,id,locations,city,state
0,11414403,"(Albuquerque, NM)",Albuquerque,NM
1,11380066,Remote,Remote,Remote
2,11375065,"(Calhoun, GA)",Calhoun,GA
3,11291995,"(Charlotte, NC)",Charlotte,NC
4,11409323,"(Bellevue, WA)",Bellevue,WA


In [41]:
locations_df.drop('locations',axis=1,inplace=True)

In [42]:
locations_df.to_csv('city_states.csv')